In [2]:
#webscraping here

import pandas as pd
import praw

reddit = praw.Reddit(client_id="69xO_MRhRBIQGprtVEZIbA",
    client_secret="GgPpkwrp_BPNduxYQ-dOcxfmrykfhQ",
    user_agent="test")


subreddit = reddit.subreddit("scu")

posts_dict = {"Body": [], "ID": [], "Ups": []}

for post in subreddit.hot(limit=1000):
    # Title and body of each post
    title = post.title
    text = post.selftext
    body = title + "\n" + text
    posts_dict['Body'].append(body)
     
    # Unique ID of each post
    posts_dict["ID"].append(post.id)
     
    # The score of a post
    posts_dict["Ups"].append(post.score)

top_posts = pd.DataFrame(posts_dict)
top_posts.to_csv("Top Posts.csv", index=True)
display(top_posts)



Version 7.7.0 of praw is outdated. Version 7.7.1 was released Tuesday July 11, 2023.


,Body,ID,Ups
0,Recruitments for Studies/Job Opportunities & O...,oouus2,18
1,Advice Paper for Incoming COEN/CS Students\nHi...,xkfy3u,46
2,Robotics and Automation in SCU\nI am from Asia...,1dbiqsj,1
3,"CRE QUESTION\nHello,\n\nI’ve took AP Calculus ...",1dbe0b2,2
4,Entire Townhome Available for Sublet//1st Quar...,1db2tb3,1
...,...,...,...
969,Freshman dorms\nHow hard is it to get into Fin...,kxpxqv,5
970,(Open to all Bay Area Residents) 1 Min Survey ...,kwq8y5,7
971,Hackathon Opportunity For SCU Students (All Ma...,ku4vj1,11
972,"Calling all engineering, computer science, and...",ks6uiv,4


In [24]:
import nltk
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

dorm_names = ['graham', 'finn', 'sanfilippo', 'campisi', 'mclaughlin-walsh', 'casa italiana', 'sobrato', 'swig', 'flip', 'camp', 'casa', 'walsh', 'university villas', 'nobili']

# Function to extract dorm names from comments
def dorm_appears(comment, dorm):
    comment_lower = comment.lower()
    tokens = tokenizer.tokenize(comment_lower)


    
    return dorm in tokens

#id_list = top_posts["ID"].tolist()
id_list = ["1d5yi68", "1cogl0"]
comments = pd.DataFrame()
comment_list = []
for i in id_list:                      #indicates start of the subission id processing
    submission = reddit.submission(i)
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        if(comment.author == "AutoModerator"):
            continue
        else:
            comment_list.append(comment.__dict__)

comments_df = pd.DataFrame(comment_list)

cleaned_df = comments_df.filter(['_submission','id','author', 'body', 'ups'], axis=1)
for dorm in dorm_names:
    dorm_column = dorm.replace(" ", "_").replace("-", "_")  # Ensure valid column names
    #display(cleaned_df['body'])
    cleaned_df[dorm_column] = cleaned_df['body'].apply(dorm_appears, dorm=dorm)

#cleaned_df.to_csv(f'comments.csv')

display(cleaned_df)

,_submission,id,author,body,ups,graham,finn,sanfilippo,campisi,mclaughlin_walsh,casa_italiana,sobrato,swig,flip,camp,casa,walsh,university_villas,nobili
0,1d5yi68,l6ostw4,Ch8ngus,Mo clue about being trans but graham and Finn ...,7,True,True,False,False,False,False,False,False,False,False,False,False,False,False
1,1d5yi68,l6oy0fr,5torm,There is no “best” dorm for LGBT people. The o...,5,False,False,False,False,False,False,False,True,False,False,True,False,False,False
2,1d5yi68,l6osuk9,MobilePiano6439,Would love to know also. I visited with my tra...,2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,1d5yi68,l6seh8v,Affectionate_Dog_600,"Swig, you will find it very inclusive there",-1,False,False,False,False,False,False,False,True,False,False,False,False,False,False
4,1d5yi68,l6oyb2n,5torm,"Correct - it’s a private, Catholic university,...",4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,1d5yi68,l6p0s3p,MobilePiano6439,I mean other similar schools do have a process...,2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,1d5yi68,l6p15hk,5torm,Fair enough - I’m sure they’ll still have a gr...,2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7,1cogl0,c9io6gg,ahcookies,Double check whether you have unpacked PluginD...,5,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8,1cogl0,c9igw5o,Nuroman,This is an issue I've been experiencing for se...,3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
9,1cogl0,c9inx72,Castun,They're Red Xs. Usually it's caused by improp...,2,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
#making csvs into what we want